https://viblo.asia/p/xay-dung-he-thong-kiem-soat-nhan-dang-khuon-mat-voi-opencv-dlib-va-deep-learning-4P8566ma5Y3?fbclid=IwAR1j2JNYg4EUr5gTy8dfynTqdOEGzUoq7d6F4Fn-0RQaMHFUjD-1K4sTf0k

In [24]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [25]:
import numpy as np
import cv2
import pickle

with open('/content/gdrive/My Drive/app/FacialRecognition/FaceNet/new1/X_train_triplet.pkl', 'rb') as f:
    X = pickle.load(f)
    X = np.array(X)
#     X = np.expand_dims(X, axis=3)
    X = X/255.
with open('/content/gdrive/My Drive/app/FacialRecognition/FaceNet/new1/y_train_triplet.pkl', 'rb') as f:
    y = pickle.load(f)
    y = np.array(y)
    
with open('/content/gdrive/My Drive/app/FacialRecognition/FaceNet/new1/name_map.pkl', 'rb') as f:
    name_map = pickle.load(f)

print(np.shape(X), np.shape(y))

(235, 221, 221, 3) (235,)


In [26]:
from keras.models import Sequential, Model
from keras.layers import *
from keras.optimizers import *
from keras import applications

import tensorflow as tf
from keras import backend as K
import h5py

def convnet_model_():
    vgg_model = applications.VGG16(weights=None, include_top=False, input_shape=(221, 221, 3))
    x = vgg_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = Lambda(lambda x_: K.l2_normalize(x,axis=1))(x)
#     x = Lambda(K.l2_normalize)(x)
    convnet_model = Model(inputs=vgg_model.input, outputs=x)
    return convnet_model

def deep_rank_model():
    convnet_model = convnet_model_()

    first_input = Input(shape=(221, 221, 3))
    first_conv = Conv2D(96, kernel_size=(8,8), strides=(16,16), padding='same')(first_input)
    first_max = MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same')(first_conv)
    first_max = Flatten()(first_max)
#     first_max = Lambda(K.l2_normalize)(first_max)
    first_max = Lambda(lambda x: K.l2_normalize(x, axis=1))(first_max)

    second_input = Input(shape=(221, 221, 3))
    second_conv = Conv2D(96, kernel_size=(8,8), strides=(32,32), padding='same')(second_input)
    second_max = MaxPool2D(pool_size=(7,7), strides=(4,4), padding='same')(second_conv)
    second_max = Flatten()(second_max)
    second_max = Lambda(lambda x: K.l2_normalize(x, axis=1))(second_max)
#     second_max = Lambda(K.l2_normalize)(second_max)
                       
    merge_one = concatenate([first_max, second_max])
    merge_two = concatenate([merge_one, convnet_model.output])
    emb = Dense(4096)(merge_two)
    emb = Dense(128)(emb)
    l2_norm_final = Lambda(lambda x: K.l2_normalize(x, axis=1))(emb)
#     l2_norm_final = Lambda(K.l2_normalize)(emb)
                        
    final_model = Model(inputs=[first_input, second_input, convnet_model.input], outputs=l2_norm_final)

    return final_model


deep_rank_model = deep_rank_model()
deep_rank_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 221, 221, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 221, 221, 64) 1792        input_4[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 221, 221, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 110, 110, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [27]:
from keras.models import load_model

print("Loading pre-trained weight")
deep_rank_model.load_weights('/content/gdrive/My Drive/app/FacialRecognition/FaceNet/new1/triplet_weight.hdf5')

from tqdm import tqdm
embs128 = []
for x in tqdm(X):
    image = x/255.
    image = np.expand_dims(image, axis=0)
    emb128 = deep_rank_model.predict([image, image, image])
    embs128.append(emb128[0])
    del image

print(np.shape(embs128))

#Create LFW_128_HA_tensor.bytes
#embs128.tofile('LFW_128_HA_tensor.bytes')

import pickle

with open('/content/gdrive/My Drive/app/FacialRecognition/FaceNet/new1/embs128.pkl', "wb") as f:
    pickle.dump(embs128, f)


#Create LFW_128_HA_labels.tsv
with open('/content/gdrive/My Drive/app/FacialRecognition/FaceNet/new1/LFW_128_HA_labels.tsv', 'w') as f:
    for label in y:
        l = str(name_map[label])
        if l == 'HA':
            l = 'Hoang Anh'
        f.write(l + '\n')

Loading pre-trained weight


100%|██████████| 235/235 [00:06<00:00, 34.22it/s]

(235, 128)
